In [1]:
# q1 -Access the NCBI database (https://www.ncbi.nlm.nih.gov/)
#  and download the DNA sequence of the Saccharomyces cerevisiae PDC gene. s)
from Bio import Entrez, SeqIO 
def download_save(acc_no, out_file):
    Entrez.email = "sargun22450@iiitd.ac.in" 
    handle = Entrez.efetch(db="nucleotide", id=acc_no, rettype="fasta", retmode="text")
    sequence = handle.read()
    
    with open(out_file, "w") as f:
        f.write(sequence)

acc_no =  "MK868028.1" 
 
out_file = "pdc.fasta" 
download_save(acc_no, out_file)

In [2]:
# q2- Use BLAST (Basic Local Alignment Search Tool) to compare the 
# PDC gene sequence across different Saccharomyces cerevisiae strains and identify SNPs
def extract_snps(blast_file):
    with open(blast_file, 'r') as file:
        lines = file.readlines()

    snps = []
    subject_sequences = {}  #to store subject seqs

    for line in lines:
        if line.startswith("Query"):
            query_sequence = line.split()[-2]
        elif line.startswith("Sbjct"):
            subj_seq = line.split()[-2]
            subj_id = line.split()[1]   
            for q, s in zip(query_sequence, subj_seq):
                if q != s or q == '-':  #if mismatch happens or there r gaps in query sequence, then 
                    position = int(line.split()[1])
                    snps.append({"position": position, "query_base": q, "subject_base": s})
            # Store subject sequence based on subject ID
            if subj_id in subject_sequences:
                subject_sequences[subj_id] += subj_seq
            else:
                subject_sequences[subj_id] = subj_seq

    # Convert subject seqs into a list
    subject_sequences_list = [{"subj_id": sid, "sequence": seq} for sid, seq in subject_sequences.items()]

    return snps, subject_sequences_list

# Example usage
blast_file = "blast_results.txt"
identified_snps, subject_sequences = extract_snps(blast_file)

# Print identified SNPs
for snp in identified_snps:
    print(f"SNP at position {snp['position']} - Query base: {snp['query_base']}, Subject base: {snp['subject_base']}")

# Print subject seqs
for subject_seq in subject_sequences:
    print(f"Subject ID: {subject_seq['subj_id']}, Sequence: {subject_seq['sequence']}")



SNP at position 233637 - Query base: G, Subject base: A
SNP at position 233517 - Query base: A, Subject base: C
SNP at position 233517 - Query base: A, Subject base: T
SNP at position 233457 - Query base: G, Subject base: A
SNP at position 233397 - Query base: A, Subject base: G
SNP at position 233277 - Query base: A, Subject base: T
SNP at position 233217 - Query base: T, Subject base: C
SNP at position 233217 - Query base: T, Subject base: C
SNP at position 233217 - Query base: G, Subject base: A
SNP at position 233217 - Query base: A, Subject base: G
SNP at position 233217 - Query base: T, Subject base: C
SNP at position 233217 - Query base: A, Subject base: T
SNP at position 233217 - Query base: T, Subject base: C
SNP at position 233157 - Query base: A, Subject base: T
SNP at position 233157 - Query base: A, Subject base: G
SNP at position 233097 - Query base: C, Subject base: A
SNP at position 233097 - Query base: A, Subject base: C
SNP at position 233097 - Query base: G, Subject 

In [3]:


# q3 Use an translation tool (e.g., ExPASy Translate tool) to translate the normal 
# and SNP-containing PDC gene sequences into amino acid sequences.

# i hv used expasy tool- 
import requests 
def translate_dna_to_protein(dna_sequence):
    url = "https://web.expasy.org/cgi-bin/translate/dna2aa.cgi"
    params = {
        'dna_sequence': dna_sequence,
        'output_format': 'fasta'
    }
    response = requests.post(url, data=params)
    return response.text.strip()

  
# translate the normal DNA sequence from the file, sends it to the ExPASy Translate tool via a POST request, 
# and retrieves the translated protein sequence.
def read_dna_seq_frmfile(fname):

    with open(fname, 'r') as file:
        next(file)
        dna_sequence = ''.join(line.strip() for line in file)

    return dna_sequence

fname = "pdc.fasta"

dna_sequence = read_dna_seq_frmfile(fname) 

protein_sequence = translate_dna_to_protein(dna_sequence) 

print("normal pdc gene translated:\n",protein_sequence)


#store in normal_pdctranslated_using_expasy file
def write_protein_sequence_to_file(protein_sequence, out_file):
    with open(out_file, 'w') as file:
        file.write(f"  Normal PDC gene translated using ExPASy\n{protein_sequence}")

out_file = "normal_pdctranslated_using_expasy.fasta"
write_protein_sequence_to_file(protein_sequence, out_file)
print(f"Translated protein sequence saved to {out_file}")


normal pdc gene translated:
 > VIRT-38133:3'5' Frame 1
LLLSVGSSSQLSLFNQVLWSIEDWQHDFNHSDLRVVVERLVLGQLVPFTG
GSDSVGFIVLSTESWQQG-VVPTLNFVVLSLWTVNQLFNGVTIVVQDEQV
WLQAPSDDGGDFLNSQLQRTVTNE-DNSLLWINFFSSESSTQGSTSGETN
GTP-DLRDTVGVVWESGLVDTESGGTGFSNDNITFLQEVTQLVPHPFLLQ
WGRSWDSSVSWSSSWDSNWLVTLSGVSNSGQQLLQNEFHLDTWESGVSDL
HVVRVEFDHCLGLVRVRERTGVEIRQQSTDRQYQISRFYSFLNFWSRQGT
NVNTTVSWVLFVNGTLTHWGVESWELSQVNQLLSFSLDVVSGTTSISQDN
WVFSILDQSQDGVNDFLFGFSIVRLQRHVNWSLQQLSWDVQVDQVSWQT-
VDLSLGDVSGSDTSVNFSWGGSNVSDHSSGFRNVGRHSVENSEVTVTQGV
VQQQLLSLSRDGWDTNNVQNTDVFSVRTGNTVQSRQFTDTEGGDDTRHTL
DTSVTISGVSSVQFVGVTSPSHTFNFVDLVQQGQVEVTWQTENGVNVDLL
-SFEQIFTQSNFRH
> VIRT-38133:3'5' Frame 2
YCLALVAAVNLACSTKFCGASKTGNMISIILILELSLKDLSWVNLSHSPV
VATLWVS-SLAPKVGNKDRWSQP-ISLY-AFGP-INFSMV-PSLFKTNKY
GFKPHLTMVEIS-TVNCKEPSPMNKITLFFGSISSAAKAAPKVAPVVKPM
EPHKT-EIP-VLFGKVVWLIPKAEVPVSAMTTSPSCKKLPNWFHIHSCFN
GVEAGTAALAGVLAGTATGL-PLAASAIVVNNFCKTNFIWTPGKVAFLIF
MWSEWNSTIVLVL-E-EKEPVLKSDNKAPIDNIKSADSTASLTSGLDKVP
T-TPPYLGCCSSMEPLPIGVSKAGN-VKSINFLVSAL

In [4]:
#in thsi func, i translate the SNP-containing PDC gene sequence from the file, send it to the ExPASy Translate tool via a POST request, 
# and then finally, retrieve the translated protein sequence.

def perform_expasy(subject_sequences): 
    protein_sequences = []  #to store protein sequences
    
    for sequence in subject_sequences:
        dna_sequence = sequence["sequence"]
        protein_sequence = translate_dna_to_protein(dna_sequence)
        print(protein_sequence) #add to lsit
        protein_sequences.append(protein_sequence) 
    
        if not protein_sequence:
            print(f"Skipping sequence {sequence['subj_id']} due to translation error")
            continue 
    
        #checking fr invalid chars
        invalid_chars = set(protein_sequence) - set("ACDEFGHIKLMNPQRSTVWY")
    
        if invalid_chars:
    
            print(f"Skipping sequence {sequence['subj_id']} if invalid: {invalid_chars}")
            continue
 

print("SNP-containing PDC gene seqs translatedd into amino acid seqs: ",protein_sequence)


perform_expasy(subject_sequences)

 

SNP-containing PDC gene seqs translatedd into amino acid seqs:  > VIRT-38133:3'5' Frame 1
LLLSVGSSSQLSLFNQVLWSIEDWQHDFNHSDLRVVVERLVLGQLVPFTG
GSDSVGFIVLSTESWQQG-VVPTLNFVVLSLWTVNQLFNGVTIVVQDEQV
WLQAPSDDGGDFLNSQLQRTVTNE-DNSLLWINFFSSESSTQGSTSGETN
GTP-DLRDTVGVVWESGLVDTESGGTGFSNDNITFLQEVTQLVPHPFLLQ
WGRSWDSSVSWSSSWDSNWLVTLSGVSNSGQQLLQNEFHLDTWESGVSDL
HVVRVEFDHCLGLVRVRERTGVEIRQQSTDRQYQISRFYSFLNFWSRQGT
NVNTTVSWVLFVNGTLTHWGVESWELSQVNQLLSFSLDVVSGTTSISQDN
WVFSILDQSQDGVNDFLFGFSIVRLQRHVNWSLQQLSWDVQVDQVSWQT-
VDLSLGDVSGSDTSVNFSWGGSNVSDHSSGFRNVGRHSVENSEVTVTQGV
VQQQLLSLSRDGWDTNNVQNTDVFSVRTGNTVQSRQFTDTEGGDDTRHTL
DTSVTISGVSSVQFVGVTSPSHTFNFVDLVQQGQVEVTWQTENGVNVDLL
-SFEQIFTQSNFRH
> VIRT-38133:3'5' Frame 2
YCLALVAAVNLACSTKFCGASKTGNMISIILILELSLKDLSWVNLSHSPV
VATLWVS-SLAPKVGNKDRWSQP-ISLY-AFGP-INFSMV-PSLFKTNKY
GFKPHLTMVEIS-TVNCKEPSPMNKITLFFGSISSAAKAAPKVAPVVKPM
EPHKT-EIP-VLFGKVVWLIPKAEVPVSAMTTSPSCKKLPNWFHIHSCFN
GVEAGTAALAGVLAGTATGL-PLAASAIVVNNFCKTNFIWTPGKVAFLIF
MWSEWNSTIVLVL-E-EKEPVLKSDNKAPIDNIKSADSTASLTSGLDKVP
T-

KeyboardInterrupt: 

In [ ]:
import requests

def run_sift_analysis(normal_sequence, snp_sequence):
    url = "https://sift.bii.a-star.edu.sg/www/SIFT_seq_submit2.html"
    params = {
        "seq": normal_sequence,
        "siftSubmit": "Submit",
        "siftseqtxt": "",
        "siftupload": "",
    }
    response_normal = requests.post(url, data=params)

    params["seq"] = snp_sequence
    response_snp = requests.post(url, data=params)

    # Process the responses and extract relevant information
    # For simplicity, we'll print the response texts
    print("Normal sequence SIFT results:")
    print(response_normal.text)

    print("\nSNP-containing sequence SIFT results:")
    print(response_snp.text)

# Normal and SNP-containing protein sequences
normal_subject_sequence = "MVHLTPVEKSAVTALWGKVNVDEVGGEALGRLLVVYPWTQRFFASFE"
snp_subject_sequence = "MVHLTPVEKSAVTALWGKVNVDEVGGEALGRLLVVYPWTQRFFGSFE"  # SNP at position 233637

# Extract normal and SNP-containing sequences around the SNP position
normal_sequence = normal_subject_sequence[233636:233639]  # Extracting 3 amino acids around the SNP
snp_sequence = snp_subject_sequence[233636:233639]  # Extracting 3 amino acids around the SNP

# Run SIFT analysis for both sequences
run_sift_analysis(normal_sequence, snp_sequence)


Normal sequence SIFT results:
<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Strict//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-strict.dtd">
<html xmlns="http://www.w3.org/1999/xhtml" xml:lang="en" lang="en">
  <head>
    <meta http-equiv="Content-Type" content="text/html; charset=utf-16" />
    <meta name="keywords" content="SIFT, non-synonymous polymorphism, nonsynonymous polymorphisms, Single Nucleotide Polymorphims analysis tool, mutation analysis tool, protein function prediction tool, amino acid substitution effect prediciton tool, human SNPs, human polymorphisms, consequence of amino acid changes, missense mutations, nsSNP, polymorphism, polymorphisms, human genetic variations, human mutagenesis, SNPs analysis tool, genome variant, genomic variation, mutation, coding nonsynonymous variants, effects of amino acid substitutions, affect protein function, deleterious AAS, Deleterious Amino Acid Substitutions, CDS variants, Damaging Amino Acid Substitutions, single nucleotide variants